In [1]:
import numpy as np
import pythonToC
import parsing_yacc 
from parsing_lex import MyLexer
import pythonTopython2
import time

In [2]:
def create_input_code(arr_len): 
    # Create a list of values with length arr_len 
    a_values = np.random.randint(0, 100, arr_len)
    a_values = a_values.tolist() 
    b_values = np.random.randint(0, 100, arr_len) 
    b_values = b_values.tolist() 

    # Create a string with the code to be executed
    # print(str(a_values)[1:-1])
    code = """
a = np.array([%s]) 
b = np.array([%s])

c = np.add(a, b) 
d = np.subtract(a, b)
e = np.sum(a) 
f = np.dot(a, b)""" % (str(a_values)[1:-1], str(b_values)[1:-1])

    return code

In [3]:
input_code = create_input_code(4) 
print(input_code)


a = np.array([72, 64, 97, 93]) 
b = np.array([19, 66, 3, 54])

c = np.add(a, b) 
d = np.subtract(a, b)
e = np.sum(a) 
f = np.dot(a, b)


In [4]:
# # Define test input
# input_code = """
# a = np.array([1, 2, 3])
# b = np.array([4, 5, 6])

# c = np.add(a, b)
# d = np.subtract(a, b)
# e = np.sum(a)
# f = np.dot(a, b)
# """

In [5]:
# Create an instance of MyLexer
lexer = MyLexer()

# Build the lexer
lexer.build()

# Test the lexer on the input code
lexer.test(input_code)

LexToken(VARIABLE,'a',1,1)
LexToken(EQUALS,'=',1,3)
LexToken(ARRAY,'np.array',1,5)
LexToken(LPAREN,'(',1,13)
LexToken(LIST,'[72, 64, 97, 93]',1,14)
LexToken(RPAREN,')',1,30)
LexToken(VARIABLE,'b',1,33)
LexToken(EQUALS,'=',1,35)
LexToken(ARRAY,'np.array',1,37)
LexToken(LPAREN,'(',1,45)
LexToken(LIST,'[19, 66, 3, 54]',1,46)
LexToken(RPAREN,')',1,61)
LexToken(VARIABLE,'c',1,64)
LexToken(EQUALS,'=',1,66)
LexToken(NP_ADD,'np.add',1,68)
LexToken(LPAREN,'(',1,74)
LexToken(VARIABLE,'a',1,75)
LexToken(COMMA,',',1,76)
LexToken(VARIABLE,'b',1,78)
LexToken(RPAREN,')',1,79)
LexToken(VARIABLE,'d',1,82)
LexToken(EQUALS,'=',1,84)
LexToken(NP_SUBTRACT,'np.subtract',1,86)
LexToken(LPAREN,'(',1,97)
LexToken(VARIABLE,'a',1,98)
LexToken(COMMA,',',1,99)
LexToken(VARIABLE,'b',1,101)
LexToken(RPAREN,')',1,102)
LexToken(VARIABLE,'e',1,104)
LexToken(EQUALS,'=',1,106)
LexToken(NP_SUM,'np.sum',1,108)
LexToken(LPAREN,'(',1,114)
LexToken(VARIABLE,'a',1,115)
LexToken(RPAREN,')',1,116)
LexToken(VARIABLE,'f',1,119)
Le

In [6]:
def deploy_parser(input, mode): 
    output = ""

    for line in input.split("\n"):
        if line == "":
            continue
        # print("line: ", line)   
        result = parsing_yacc.main(line, mode)
        # print("result: ", result)
        # Check if the result is a string 
        if isinstance(result, str):
            output += result + "\n"

    return output

In [7]:
# Generate C code from Python input using yacc
print("Generating C code...")
#measure time to genreate C code - 
start_time = time.time()

c_output = deploy_parser(input_code, "C")
print("C code generation complete.")

c_time  = time.time() - start_time
print("Time taken to generate C code:", c_time)
print("\n")

print("c_output:", c_output)

if c_output:
    # Set up files for conversion
    pythonToC.set_up("test")
    pythonTopython2.setup("test")

    # Execute the input code to define variables
    exec(input_code)

    # Generate C code from Python input using pythontoC functions
    for line in input_code.split('\n'):
        if '=' in line:
            variable, value = map(str.strip, line.split("="))
            if '[' in value:  # Check if value is an array
                array_length = len(eval(value))  # Evaluate the array and get its length
                pythonToC.set_arr(variable, value, array_length, "test")
            else:  # Single value
                pythonToC.set_arr(variable, value, 1, "test")



    # Generate Python code from Python input using pythontopython functions
    for line in input_code.split('\n'):
        if '=' in line:
            exec("pythonTopython2." + line.strip())
    p_time  = time.time() - start_time
    

    # Generate Python code from C input using pythontopython functions
    #for line in c_output.split('\n'):
        #if line.strip():
            #exec("pythonTopython2." + line.strip())

#     print("Conversion complete.")
# else:
#     print("No C output generated.")


Generating C code...
C code generation complete.
Time taken to generate C code: 0.00404810905456543


c_output:    float h_a[72, 64, 97, 93] = [72, 64, 97, 93];
    float *d_a;
   cudaMalloc(&d_a,72, 64, 97, 93*sizeof(float);
    cudaMemcpy(d_a, h_a,sizeof(h_a), cudaMemcpyHostToDevice);


   float h_b[19, 66, 3, 54] = [19, 66, 3, 54];
    float *d_b;
   cudaMalloc(&d_b,19, 66, 3, 54*sizeof(float);
    cudaMemcpy(d_b, h_b,sizeof(h_b), cudaMemcpyHostToDevice);


addArraysHelper(a, b, c);
subtractArraysHelper(a, b, d);
sumArraysHelper(a, e);

Conversion complete.


In [6]:
# Generate Python code from Python input using yacc
print("Generating Python code...")
start_time = time.time()

python_output = deploy_parser(input_code, "Python")
print("Python code generation complete.")

p_time  = time.time() - start_time
print("Time taken to generate Python code:", p_time)
print("\n")

print("python_output:", python_output)

if python_output:
    pass 


Generating Python code...
Python code generation complete.
Time taken to generate Python code: 0.0020599365234375


python_output: a = cupy.array([1, 2, 3])
b = cupy.array([4, 5, 6])
cupy.add(a, b)
cupy.subtract(a, b)
cupy.sum(a)
cupy.dot(a, b)

